## Trabajo Fin de Máster <br/> Diseño de una arquitectura multimodal para descripción textual de pares imagen-audio

## Script 1. Preprocesamiento de audio usando PyTorch

En este notebook, creamos una base de datos para un clasificador de audio, a partir de unos pocos audios originales. Para ello, usamos lo compartido en el artículo de [TowardsDataScience](https://towardsdatascience.com/audio-deep-learning-made-simple-sound-classification-step-by-step-cebc936bbe5). Tomamos las clases que se definen en dicho artículo, y las enriquecemos con métodos propios, y otros extraídos de esta entrada en [medium](https://medium.com/@alibugra/audio-data-augmentation-f26d716eee66).


### Paso 1. Lectura de audios originales

Accedemos a la carpeta donde se encuentran los audios originales y generamos un dataframe con información básica sobre ellos.

In [1]:
import os
import shutil
import pandas as pd

In [2]:
os.chdir("..")
os.getcwd()

'/mnt/batch/tasks/shared/LS_root/mounts/clusters/tfm-cpu/code/Users/jose.puche/Scripts'

In [8]:
from tfm_lib.audio_processing import AudioUtil, AudioAugmentation

def audio_files_df(database_path):

    participants = os.listdir(database_path)

    rel_paths = []
    sample_rates = []
    channels = []
    duration = []
    classID = []

    for person in participants:
        if '.' in person:
            continue
        relative_path = f"{database_path}/{person}/audio.ogg"
        sig, sr = AudioUtil.open(relative_path)
            

        rel_paths.append(relative_path)
        sample_rates.append(sr)
        channels.append(sig.shape[0])
        duration.append(sig.shape[1]/sr)
        classID.append(person)

    return pd.DataFrame({
        "relative_path": rel_paths,
        "sample_rate": sample_rates,
        "channels": channels,
        "duration (s)": duration,
        "classID": classID
    })


df = audio_files_df('../Database')
display(df)

,relative_path,sample_rate,channels,duration (s),classID
0,../Database/Alba Azorin Zafrilla/audio.ogg,16000,1,23.713500,Alba Azorin Zafrilla
1,../Database/Alfonso Girona Palao/audio.ogg,16000,1,21.933500,Alfonso Girona Palao
2,../Database/Alfonso Vidal Lopez/audio.ogg,16000,1,22.993500,Alfonso Vidal Lopez
3,../Database/Ana Azorin Puche/audio.ogg,16000,1,21.433500,Ana Azorin Puche
4,../Database/Ana Puche Palao/audio.ogg,16000,1,25.537813,Ana Puche Palao
5,../Database/Angela Espinosa Martinez/audio.ogg,48000,1,25.980333,Angela Espinosa Martinez
6,../Database/Clara Hidalgo Lopez/audio.ogg,16000,1,21.917813,Clara Hidalgo Lopez
7,../Database/Cristina Carpena Ortiz/audio.ogg,16000,1,25.993500,Cristina Carpena Ortiz
8,../Database/David Azorin Soriano/audio.ogg,16000,1,21.757812,David Azorin Soriano
9,../Database/Diego Molina Puche/audio.ogg,16000,1,22.657812,Diego Molina Puche


### Paso 3. Creación de la base de datos de audio

En este paso, leemos cada uno de los audios que conocimos anteriormente y aplicamos sobre ellos una serie de transformaciones para homogeneizarlos, y también para crear audios nuevos a partir de los originales.

Las transformaciones son las siguientes:

* **Limpieza.** Podamos el inicio y final del audio, puesto que la mayoría de participantes comienzan y acaban su grabación con unos instantes de silencio (no nos interesa que la red aprenda silencios que no formen parte del discurso leído).

* **Resampleado.** Nos aseguramos de que la frecuencia de muestreo de todos los audios originales sea la misma (modificándola en caso de que sea necesario).

* **Igualar número de canales.** Dado un número deseado de canales (1 ó 2), nos quedamos con el primer canal (para pasar de estéreo a mono) o duplicamos el canal existente (de mono a estéreo).

* **Fragmentación.** Cortamos el audio en una serie de trozos de una duración determinada. En este caso se multiplica el número de audios de 5 a 10 veces.

Una vez tenemos estos trozos de audios originales, generemos audios completamente nuevos con técnicas de Data Augmentation:

* **Inserción de ruido.** Distorsionamos de forma aleatoria la señal.

* **Tralación temporal.** Pegamos el inicio y final del audio y trasladamos el inicio a un instante distinto del original.

Por medio de estas trasnformaciones, multiplicamos por 5 el número de audios que habíamos obtenido en la fragmentación.

In [11]:
def ExtendDatabase(db_path, new_db_path, sample_rate, channels, duration):

    # For each name in the list of participants, we apply the same transformations
    # and create a new folder in the new database with all the new audios
    participants = os.listdir(db_path)
    participants.remove(".DS_Store")

    classID_train = []
    rel_paths_train = []
    classID_test = []
    rel_paths_test = []
    for person in participants:
        if '.' in person:
            continue
        relative_path = f"{db_path}/{person}/audio.ogg"
        aud = AudioUtil.open(relative_path)

        # First, we clean and standarize the audios
        cln_aud = AudioUtil.prune_audio(aud)
        reaud = AudioUtil.resample(cln_aud, sample_rate)
        rch_aud = AudioUtil.rechannel(reaud, channels)

        # Now, we split the audio that results and save the results
        lst_aud_0 = AudioUtil.split_audio(rch_aud, duration=duration)
        lst_aud_1 = list(map(AudioAugmentation.add_noise, lst_aud_0))
        lst_aud_2 = list(map(AudioAugmentation.time_shift, lst_aud_0))
        lst_aud_3 = list(map(AudioAugmentation.add_noise, lst_aud_2))
        lst_aud_4 = list(map(AudioAugmentation.time_shift, lst_aud_2))
        lst_aud_5 = list(map(AudioAugmentation.add_noise, lst_aud_4))
        lst_aud_6 = list(map(AudioAugmentation.time_shift, lst_aud_4))
        lst_aud_7 = list(map(AudioAugmentation.add_noise, lst_aud_6))
        lst_aud_8 = list(map(AudioAugmentation.time_shift, lst_aud_6))
        lst_aud_9 = list(map(AudioAugmentation.add_noise, lst_aud_8))

        # Remove the new database folder in case it exists
        # Create the empty folder with the same name
        folder_path = f'{new_db_path}/audio/{person}'
        if os.path.exists(folder_path):
            shutil.rmtree(folder_path)
        os.makedirs(folder_path)

        for k in range(6):
            var_name = f"lst_aud_{k}"
            for i, audio in enumerate(vars()[var_name]):
                path = f'{folder_path}/audio{str(k).zfill(2)}{str(i).zfill(2)}.ogg'
                _ = AudioUtil.save(audio, path)
                classID_train.append(person)
                rel_paths_train.append(path)

        for k in range(6,10):
            var_name = f"lst_aud_{k}"
            for i, audio in enumerate(vars()[var_name]):
                path = f'{folder_path}/audio{str(k).zfill(2)}{str(i).zfill(2)}.ogg'
                _ = AudioUtil.save(audio, path)
                classID_test.append(person)
                rel_paths_test.append(path)


    return pd.DataFrame({ 'audio_path': rel_paths_train, 'classID': classID_train }), \
           pd.DataFrame({ 'audio_path': rel_paths_test, 'classID': classID_test })

train_df, test_df = ExtendDatabase('../Database', '../Final_Database', 16000, 1, 4)
train_df.to_csv("../Final_Database/audio/audioDB_train.csv", index=False)
test_df.to_csv("../Final_Database/audio/audioDB_test.csv", index=False)

In [ ]:
database_df = pd.read_csv("/content/drive/MyDrive/TFM/Proyecto/Final_Database_mini/audio/audioDB.csv")
database_df.head()

,audio_path,classID
0,../Final_Database_mini/audio/Jose Alberto Azor...,Jose Alberto Azorin Puche
1,../Final_Database_mini/audio/Jose Alberto Azor...,Jose Alberto Azorin Puche
2,../Final_Database_mini/audio/Jose Alberto Azor...,Jose Alberto Azorin Puche
3,../Final_Database_mini/audio/Jose Alberto Azor...,Jose Alberto Azorin Puche
4,../Final_Database_mini/audio/Jose Alberto Azor...,Jose Alberto Azorin Puche
